In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from time import sleep
from tqdm import tqdm
import json
import math

In [ ]:
API_KEY1 = "91ed49cd-f2bb-4bba-9ff8-0f4ac4584790"
API_KEY2 = "эээ ну короче нужен запасной ключ" 

In [9]:
BOX = {
    "Москва": (37.30, 37.95, 55.55, 55.95),
    "Санкт-Петербург": (30.10, 30.50, 59.80, 60.10)
}

CITIES = [
    "Москва", "Санкт-Петербург"
]

In [10]:
cities_df = pd.DataFrame(columns=["city", "lon_min", "lon_max", "lat_min", "lat_max"])
i=0

for city in CITIES:
  lon_min, lon_max, lat_min, lat_max = BOX[city]
  cities_df.loc[i] = [city, lon_min, lon_max, lat_min, lat_max]
  i+=1

cities_df.to_csv("cities.csv", index=False)
print("Сохранено как 'cities.csv'")


Сохранено как 'cities.csv'


In [11]:
cities_df = pd.read_csv("cities.csv")
cities_df

,city,lon_min,lon_max,lat_min,lat_max
0,Москва,37.3,37.95,55.55,55.95
1,Санкт-Петербург,30.1,30.50,59.80,60.10


In [ ]:
import logging
# тут все аналогично первому логгеру из обычной апишки, но уровень ставим debug тк будет очень много несущественных оповещений, а так же задаем другой логгер - 2GIS_API.api
logger = logging.getLogger("2GIS_API.api")
logger.setLevel(logging.DEBUG)

fh2 = logging.FileHandler("Metro_APILogs.log")

formatter2 = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh2.setFormatter(formatter2)

logger.addHandler(fh2)

In [ ]:
# --- настройки под демо-лимиты ---
PAGE_SIZE = 10      # демо: 1-10
MAX_PAGES = 5       # демо: 5 страниц на точку
RADIUS_M = 3000     # радиус поиска в метрах
METERS_PER_DEG_LAT = 111_320.0  # ~м/градус широты

def rotate_key(req_count: int, current_key: str) -> str:
    # если по какой-то причине слишком много запросов, например много сетевых ошибок, то меняем ключ на запасной
    if req_count >= 999 and current_key == API_KEY1:
        logger.warning("Changed the keys")
        return API_KEY2
    return current_key


def fetch_circle(lon: float, lat: float, city_id: str, req_count: int, api_key: str):
    out = []
    for page in range(1, MAX_PAGES + 1):  # перебираем страницы для обращения к API
        api_key = rotate_key(req_count, api_key)  # проверяем и меняем ключ если необходимо

        for attempt in range(3):  # пробуем три раза на случай различных несущественных ошибок
            try:
                r = requests.get(
                    "https://catalog.api.2gis.com/3.0/items",
                    params={
                        "key": api_key,
                        "q": "метро",
                        "type": "station.metro",
                        "city_id": city_id,
                        "point": f"{lon},{lat}",
                        "radius": RADIUS_M,
                        "fields": "items.point,items.name",
                        "page": page,
                        "page_size": PAGE_SIZE,
                    },
                    timeout=10,
                )
                req_count += 1  # увеличиваем текущий счетчик запросов

                r.raise_for_status()  # далее как в предыдущем коде обработка ошибок
                data = r.json()
                code = data["meta"]["code"]
                
                if code == 200:
                    logger.debug("Code 200")
                    break
                else:
                    message = data["meta"]["error"]["message"]
                    if code == 404:  # в выбранном секторе не обязательно есть станции метро
                        logger.warning(f"API request returned error code {code}, message: '{message}' - probably no subway in this area")
                    elif code != 200:
                        # если проблема, например с загрузкой, то пробуем еще раз
                        wait_s = attempt + 1
                        logger.error(f"API request error: code {code}, message: '{message}'. Waiting {wait_s} s and retrying, attempt {attempt+1}/3")
                        sleep(wait_s)
                        continue
                    else:
                        logger.error(f"Unexpected API response error: code {code}, message: '{message}'")
                    break

            except requests.exceptions.HTTPError as e:
                logger.error(f"HTTP error during API request: {e}")
            except Exception as e:
                logger.error(f"Unexpected error during API request: {e}")
        else:
            # все попытки провалились - выходим из пагинации этого круга
            break

        if code == 200:  # если код 200, то получаем и записываем нужные нам результаты
            items = data["result"]["items"]
            if not items:
                break

            out.extend(items)
            # если предметов меньше чем мы запросили, то, вероятно, следующая страница будет пустой
            if len(items) < PAGE_SIZE:
                break
        elif code == 404:  # если код 404, те нет станций метро в этой области, то нет смысла листать страницы
            break
        sleep(0.15)

    return out, req_count, api_key


#-------------------------------------------------------#
#                       начало                          #
#-------------------------------------------------------#


logger.info("Started")

# шаг в градусах по широте эквивалентный RADIUS_M км
distlat_deg = RADIUS_M / METERS_PER_DEG_LAT
req_count = 0
api_key = API_KEY1

frames = []  # сюда будем добавлять то что получили из API

for i in range(cities_df.shape[0]):
    city, lon_min, lon_max, lat_min, lat_max = cities_df.iloc[i]
    city_id = "4504222397630173" if city == "Москва" else "5348647327760881"

    logger.info(f"Processing city: {city}. API requests so far: {req_count}.")

    centers = []  # тут определяем центры как было расписано в ГП_API
    row = 0
    lat = lat_min
    while lat <= lat_max + 1e-9:
        distlon_deg = 2 * RADIUS_M / (METERS_PER_DEG_LAT * math.cos(math.radians(lat)))
        start_lon = lon_min + (0.5 * distlon_deg if (row % 2) else 0.0)
        lon = start_lon
        while lon <= lon_max + 1e-9:
            centers.append((lat, lon))
            lon += distlon_deg
        row += 1
        lat += distlat_deg

    logger.debug(f"Number of centers for city '{city}': {len(centers)}")

    collected = {}  # сюда складываем временные данные, которые получим после прохода по всем центрам города
    j = 0
    for (lat, lon) in centers:
        logger.debug(f"Processing center #{j}, {len(centers) - j} centers remaining")
        j += 1
        # Важно: порядок аргументов point это lon,lat
        objs, req_count, api_key = fetch_circle(lon, lat, city_id, req_count, api_key)

        logger.debug(f"Current request count: {req_count}")
        for it in objs:
            if isinstance(it, dict) and "id" in it:
                # то что мы получили из апишки мы записываем в формате id: {...}
                collected[it["id"]] = it

    df_city = pd.json_normalize(list(collected.values()))
    if not df_city.empty:
        df_city["city"] = city  # записываем город
        frames.append(df_city)  # сохраняем это в список frames

df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
df.to_csv("api_metro_stations.csv", index=False)
print("CSV сохранен: api_metro_stations.csv")
logger.info("Finished")
